In [1]:
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None,*,value = None):
        self.feature = feature #which feature was devived with
        self.threshold = threshold #threshold value
        self.left = left #left child
        self.right = right #right child
        self.value = None #value of the node

        def is_leaf_node(self):
            return self.value is not None


In [3]:
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from collections import Counter

class DecisionTree:
    def  __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=5, random_state=0, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state #seed
        self.criterion = criterion
        self.root = None   # root of the tree 

    def _most_common_label(self, y):
        #leggyakoribb label meghatarozasa
        counter = Counter(y)
        return counter.most_common(1)[0][0]
    
    def _best_split(self, X, y, feat_idxs):
        #legjobb elvalasztas meghatarozasa
        best_gain = -np.inf
        split_idx, split_treshhold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:,feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                #calculate information gain
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_treshhold = threshold

        return split_idx, split_treshhold

    def _grow_tree(self, X, y, depth=0):
        number_of_samples, number_of_features = X.shape
        number_of_labels = len(np.unique(y))
        #check stopping criteria
        if(depth >= self.max_depth | number_of_labels == 1 | number_of_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        #find best split
        feat_idxs = np.random.choice(number_of_features, self.random_state, replace=False)
        best_featuree, best_treshhold = self._best_split(X, y, feat_idxs)
        #create children nodes

        #recursively call the function

        return None


    def fit(self, X, y):
        #tbp
        #döntési fa tanítása
        self.root = self._grow_tree(X, y)

        return None
    
    def predict(self, X):
        #tbp
        #regresszios ertek meghatarozasa
        return None
    
    def print():
        #döntési fa ábrázolása
        #tbp
        return None

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

#Minta adatforrás
wine = load_wine()
X = wine.data # adatok
y = wine.target # címkék/célváltozók

#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő

#Döntési fa tanítása
dt = DecisionTree()
dt.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = dt.predict(X_test)
print(y_pred)

dt.print()